First, install the gegravity package.

In [ ]:
pip install gegravity

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 1.5 MB/s eta 0:00:00


LOAD THE PACKAGES

In [ ]:
import gegravity as ge
import pandas as pd
# Increase number of columns printed for a pandas DataFrame
pd.set_option("display.max_columns", None)
pd.set_option('display.width', 1000)
import gme as gme

Next, load the gravity data

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving DEBASHREE_FINALDATASET_200307.csv to DEBASHREE_FINALDATASET_200307.csv


In [ ]:
gravity_data_location = "/content/DEBASHREE_FINALDATASET_200307.csv"
grav_data = pd.read_csv(gravity_data_location)
df = pd.read_csv('/content/DEBASHREE_FINALDATASET_200307.csv')
print(grav_data.head())

   year iso_o iso_d   dist    lndist      trade_o  asean_FTA  contig  comlang_off  comcol  comrelig     gdp_o         gdp_d  rta  home         flow  asean   tariff  reporter_prevalence_score  partner_prevalence_score  international  concat
0  2021   ARE   ARE    109  4.691348        0.000          0       0            0       0  0.900626  41500000  4.150000e+07    0     1        0.000      0  0.00000                        9.0                       0.0              0  AREARE
1  2021   ARE   AUS  11658  9.363748   625480.030          0       0            0       0  0.003787  41500000  1.540000e+09    0     0   625480.030      0  4.48444                        3.4                       3.5              1  AREAUS
2  2021   ARE   BRN   6734  8.814925    26570.818          0       0            0       1  0.609419  41500000  1.400000e+07    0     0    26570.818      0  4.80238                        3.4                       2.4              1  AREBRN
3  2021   ARE   CHN   6430  8.768730  92

Prepare data and econometric inputs for GE model

In [ ]:
# Define GME Estimation Data
gme_data = gme.EstimationData(grav_data, # Dataset
                              imp_var_name="iso_d", # Importer column name
                              exp_var_name="iso_o", # Exporter column name
                              year_var_name = "year",  # Year column name
                              trade_var_name="trade_o")  # Trade column name

In [ ]:
#Create Gravity Model
gme_model = gme.EstimationModel(gme_data, # Specify data to use
                                lhs_var="trade_o",                               # dependent, "left hand side" variable
                                rhs_var=[ "tariff","contig","comlang_off","comcol", # independent variables
                                         "lndist", "asean_FTA","international"],
                                fixed_effects=[["iso_o"],["iso_d"]])     # Fixed effects to use

In [ ]:
# Estimate gravity model with PPML
gme_model.estimate()

Estimation began at 10:36 AM  on Nov 17, 2023
Omitted Regressors: ['iso_d_fe_VNM']
Estimation completed at 10:36 AM  on Nov 17, 2023


/usr/local/lib/python3.10/dist-packages/gme/estimate/_ppml_estimation_and_diagnostics.py:254: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  adjusted_data_frame.drop(col, 1, inplace=True)
/usr/local/lib/python3.10/dist-packages/gme/estimate/_ppml_estimation_and_diagnostics.py:306: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  diagnostics = overfit_column.append(exclusion_column)


{'all': <statsmodels.genmod.generalized_linear_model.GLMResultsWrapper at 0x7f65de96f190>}

In [ ]:
# Print econometric results table
print(gme_model.results_dict['all'].summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:                trade_o   No. Observations:                  256
Model:                            GLM   Df Residuals:                      218
Model Family:                 Poisson   Df Model:                           37
Link Function:                    Log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:            -3.3422e+08
Date:                Fri, 17 Nov 2023   Deviance:                   6.6844e+08
Time:                        10:36:52   Pearson chi2:                 7.10e+08
No. Iterations:                    14   Pseudo R-squ. (CS):              1.000
Covariance Type:                  HC1                                         
                    coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
tariff           -0.0533      0.062     -0.857

After econometric estimation, now we solve the baseline model. We assume ARE(United Arab Emirates) is the reference importer, and take cross section sample for year 2021


In [ ]:
# Define GE model
ge_model = ge.OneSectorGE(gme_model,                   # gme gravity model
                       year = "2021",               # Year to use for model
                       expend_var_name = "gdp_d",       # Expenditure column name
                       output_var_name = "gdp_o",       # Output column name
                       reference_importer = "ARE",  # Reference importer
                       sigma = 5)                   # Elasticity of substitution

The following commands are not required to define or solve the GE model but can help diagnose issues that arise if
the model fails to solve.

In [ ]:
# Test that the model system of equations is computable from the supplied data and parameters
test_diagnostics = ge_model.test_baseline_mr_function()
# See what is returned:
print(test_diagnostics.keys())
# Check the values of the model parameters computed from the baseline data, which should be numeric with no missing
# values
input_params = test_diagnostics['mr_params']
# Check one set of parameters, for example:
print(input_params['cost_exp_shr'])

dict_keys(['initial values', 'mr_params', 'function_value'])
[[9.79364792e-03 4.32745306e-04 5.47783956e-01 3.74874798e-02
  9.70590055e-02 1.51498463e-01 8.04825595e-04 5.78991645e-04
  1.96209761e-03 1.16095052e-02 1.25377384e-02 1.27904663e-02
  1.55152057e-02 6.82032208e-01 1.11594979e-02 8.84763207e-01]
 [5.61696010e-02 1.66932108e-04 6.82467168e-01 6.60647606e-02
  4.21758691e-01 1.84769185e-01 1.54777492e-03 1.04002867e-03
  9.34755846e-03 7.51359656e-02 2.23154246e-02 5.94264059e-02
  2.76678866e-02 7.70466968e-01 2.07674943e-02 2.91402967e+00]
 [3.79525621e-02 4.19338138e-04 1.11498821e-01 3.11409656e-02
  1.12012226e-01 1.39526062e-01 7.44526966e-04 7.05729219e-04
  2.33409526e-03 1.01861455e-02 1.12911462e-02 1.09693846e-02
  1.36077930e-02 5.42340911e-01 1.31902177e-02 7.93268224e-01]
 [3.56273583e-02 5.28592949e-04 4.27892375e-01 1.09550030e-02
  1.03263559e-01 1.15122899e-01 6.68318620e-04 5.91023565e-04
  1.65449688e-03 1.83334865e-02 1.29499828e-02 1.48720642e-02
  1.75

Check scaling of outward multilateral resistances (OMRs)

In [ ]:
# Check for OMR rescale factors that results in convergence
rescale_eval = ge_model.check_omr_rescale(omr_rescale_range=3)
print(rescale_eval)


Trying OMR rescale factor of 0.001
Solving for baseline MRs...
The solution converged.

Trying OMR rescale factor of 0.01
Solving for baseline MRs...
The solution converged.

Trying OMR rescale factor of 0.1
Solving for baseline MRs...
The solution converged.

Trying OMR rescale factor of 1
Solving for baseline MRs...
The solution converged.

Trying OMR rescale factor of 10
Solving for baseline MRs...
The solution converged.

Trying OMR rescale factor of 100
Solving for baseline MRs...
The solution converged.

Trying OMR rescale factor of 1000
Solving for baseline MRs...
The solution converged.
   omr_rescale omr_rescale (alt format)  solved                  message  max_func_value  mean_func_value  reference_importer_omr
0        0.001                    10^-3    True  The solution converged.    9.801919e-09     1.828535e-10                0.786262
1        0.010                    10^-2    True  The solution converged.    1.310770e-10    -5.477840e-12                0.786262
2      

The next set of steps would involve solving baseline and experiment GE model.

In [ ]:
# Solve the baseline model
ge_model.build_baseline(omr_rescale=100)
# Examine the solutions for the baseline multilateral resistances
print(ge_model.baseline_mr.head(16))

Solving for baseline MRs...
The solution converged.
         baseline omr  baseline imr
country                            
ARE          0.786262      1.000000
AUS          0.744974      1.024965
BRN          0.634357      0.985064
CHN          0.809144      1.076466
IDN          0.794634      1.016907
IND          0.741548      0.979618
JPN          0.750994      1.019591
KHM          0.818215      1.022609
LAO          0.768698      0.990892
MMR          0.650138      0.977245
MYS          0.678582      0.976890
PHL          0.760701      1.000332
SGP          0.633560      0.975075
THA          0.808777      1.006177
USA          0.822774      1.165354
VNM          0.782769      0.978487


Now, till here were the codes to solve the baseline gravity model, now we can use the solved baseline model to conduct various counterfactual experiments. As per our research objectives, the counterfactual is adding Australia into ASEAN. For this purpose, we change asean_FTA with all member countries as 1. So, now we define the counterfactual experiment.

In [ ]:
# Create a copy of the baseline data
exp_data = ge_model.baseline_data.copy()

In [ ]:
#We develop an array of all member countries of ASEAN.
member_countries = ["BRN", "IDN", "IND", "KHM", "LAO", "MYS", "MMR", "PHL", "SGP", "THA", "VNM"]

#For our counterfactual experiment, we set the value of asean_FTA dummy variable with all the member countries as 1. We use 'for' loop for this iterative purpose.
for country in member_countries:
  exp_data.loc[(exp_data["iso_d"] == "AUS") | (exp_data["iso_o"] == country), "asean_FTA"] = 1
  exp_data.loc[(exp_data["iso_d"] == country) | (exp_data["iso_o"] == "AUS"), "asean_FTA"] = 1

# Define the experiment within the GE model
ge_model.define_experiment(exp_data)

# Examine the baseline and counterfactual trade costs
print(ge_model.bilateral_costs.head(56))

             baseline trade cost  experiment trade cost  trade cost change (%)
iso_o iso_d                                                                   
ARE   ARE               0.656763               0.656763                0.00000
      AUS               2.303839               3.132508               35.96903
      BRN               6.293561               8.557294               35.96903
      CHN               2.346110               2.346110                0.00000
      IDN               2.367082               3.218498               35.96903
      IND               6.832162               9.289624               35.96903
      JPN               2.364943               2.364943                0.00000
      KHM               2.362041               3.211644               35.96903
      LAO               2.361185               3.210480               35.96903
      MMR               6.589376               8.959511               35.96903
      MYS               6.458942               8.782

In [ ]:
# Simulate the counterfactual model
ge_model.simulate()

Solving for conditional MRs...
The solution converged.
Solving full GE model...
The solution converged.


In [ ]:
# We can examine the counterfactual trade flows predicted by the model.
print(ge_model.bilateral_trade_results.head(50))

             baseline modeled trade  experiment trade  trade change (percent)
iso_o iso_d                                                                  
ARE   ARE              5.937136e+06      5.840331e+06               -1.630495
      AUS              1.155669e+06      1.251077e+06                8.255630
      BRN              2.448533e+04      2.873616e+04               17.360722
      CHN              1.645690e+07      1.623509e+07               -1.347823
      IDN              8.890153e+05      9.829551e+05               10.566726
      IND              5.886648e+06      6.516330e+06               10.696787
      JPN              3.726282e+06      3.667197e+06               -1.585627
      KHM              2.058325e+04      2.276869e+04               10.617546
      LAO              1.263042e+04      1.405698e+04               11.294611
      MMR              1.154684e+05      1.269860e+05                9.974710
      MYS              6.475560e+05      7.432372e+05           

Now that our baseline and counterfactual models are solved, we can access and export the  GE results. We retrieve many of the different sets of model results.

In [ ]:
# A collection of many of the key country-level results (prices, total imports/exports, GDP, welfare, etc.)
country_results = ge_model.country_results

In [ ]:
print(country_results)

         factory gate price change (percent)  omr change (percent)  imr change (percent)  GDP change (percent)  welfare statistic  terms of trade change (percent)  output change (percent)  expenditure change (percent)  foreign exports change (percent)  foreign imports change (percent)  intranational trade change (percent)
country                                                                                                                                                                                                                                                                                                                            
ARE                                 0.232614             -0.232074              0.000000              0.232614           0.997679                         0.232614                 0.232614                      0.232614                          0.559413                          0.305377                             -1.630495
AUS                         

In [ ]:
# The bilateral trade results
bilateral_results = ge_model.bilateral_trade_results

In [ ]:
print(bilateral_results.head(60))

             baseline modeled trade  experiment trade  trade change (percent)
iso_o iso_d                                                                  
ARE   ARE              5.937136e+06      5.840331e+06               -1.630495
      AUS              1.155669e+06      1.251077e+06                8.255630
      BRN              2.448533e+04      2.873616e+04               17.360722
      CHN              1.645690e+07      1.623509e+07               -1.347823
      IDN              8.890153e+05      9.829551e+05               10.566726
      IND              5.886648e+06      6.516330e+06               10.696787
      JPN              3.726282e+06      3.667197e+06               -1.585627
      KHM              2.058325e+04      2.276869e+04               10.617546
      LAO              1.263042e+04      1.405698e+04               11.294611
      MMR              1.154684e+05      1.269860e+05                9.974710
      MYS              6.475560e+05      7.432372e+05           

In [ ]:
# A wider selection of aggregate, country-level trade results
agg_trade = ge_model.aggregate_trade_results
print(agg_trade)

         baseline modeled shipments  experiment shipments  shipments change (percent)  baseline modeled foreign exports  experiment foreign exports  foreign exports change (percent)  baseline modeled consumption  experiment consumption  consumption change (percent)  baseline modeled foreign imports  experiment foreign imports  foreign imports change (percent)  baseline modeled intranational trade  experiment modeled intranational trade  intranational trade change (percent)
country                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      
ARE                    6.512166e+07         

In [ ]:
# country multilateral resistance (MR) terms
mr_terms = ge_model.country_mr_terms
# Get the solver diaganoistics, which is a dictionary containing many types of solver diagnostic info
solver_diagnostics = ge_model.solver_diagnostics
mr_terms = ge_model.country_mr_terms
print(mr_terms)

         baseline imr  conditional imr  experiment imr  baseline omr  conditional omr  experiment omr
country                                                                                              
ARE          1.000000         1.000000        1.000000      0.786262         0.784437        0.784437
AUS          1.024965         0.955050        0.955050      0.744974         0.692229        0.692229
BRN          0.985064         0.938667        0.938667      0.634357         0.594692        0.594692
CHN          1.076466         1.077513        1.077513      0.809144         0.808087        0.808087
IDN          1.016907         0.953825        0.953825      0.794634         0.742306        0.742306
IND          0.979618         0.918304        0.918304      0.741548         0.690264        0.690264
JPN          1.019591         1.020012        1.020012      0.750994         0.750147        0.750147
KHM          1.022609         0.959974        0.959974      0.818215         0.766

In [ ]:
solver_diagnostics = ge_model.solver_diagnostics

In [ ]:
# Export the results to a collection of spreadsheet (.csv) files and add trade values in levels to the outputs.
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
output_directory = "/content/drive/My Drive/"
os.makedirs(output_directory, exist_ok=True)
ge_model.export_results(directory=output_directory, name="GE_analysis")